In [1]:
import pandas as pd
import numpy as np
import os
from cleantext import clean
from collections import defaultdict
import re
import sys
import argparse
from pathlib import Path

# Example command:
# python identify_paragraphs_containing_keywords.py C:\Users\jasonjia\Dropbox\Projects\conference_call\output\02_process_cc\02.2_csv_20210101_20220617 C:\Users\jasonjia\Dropbox\Projects\conference_call\output\03_identify_paragraphs_containing_keywords C:\Users\jasonjia\Dropbox\Projects\conference_call\code\03_identify_paragraphs_containing_keywords\reference_files\keywords.txt

In [2]:
# Set the max number of columns and rows to display in Jupyter Notebooks
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [3]:
inputfolder = Path(r"C:\Users\jasonjia\Dropbox\Projects\conference_call\output\02_process_cc\02.2_csv_20210101_20220617")
outputfolder = Path(r"C:\Users\jasonjia\Dropbox\Projects\conference_call\output\03_identify_paragraphs_containing_keywords")
keywords_filepath = Path(r"C:\Users\jasonjia\Dropbox\Projects\conference_call\code\03_identify_paragraphs_containing_keywords\reference_files\keywords.txt")

In [ ]:
# Parse arguments
if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Identify paragraphs containing keywords')
    parser.add_argument('inputfolder', help="input folder containing the .csv files", type=str)
    parser.add_argument('outputfolder', help="output folder containing the paragraphs", type=str)
    parser.add_argument('keywords_filepath', help="filepath of the keywords.txt file", type=str)
    args = parser.parse_args()
    inputfolder = Path(args.inputfolder)
    outputfolder = Path(args.outputfolder)
    keywords_filepath = Path(args.keywords_filepath)

print("Input folder:", inputfolder)
print("Output folder:", outputfolder)
print("Keywords filepath:", keywords_filepath)

# Functions

In [71]:
def clean_str(s):
    t = s.replace("\\n", " ")
    t = t.replace("\\", "")
    if len(t) == 0:
        return ""
    if t[0] == '"':
        t = t[1:-1]
    return clean(t, fix_unicode = True, no_line_breaks = True, lower = False)

In [72]:
def has_numbers(inputString):
    return bool(re.search(r'\d', inputString))

In [73]:
def extractNumbers(text):
    digits = re.findall(r"[(\d.)]+", text)
    digits = [i for i in digits if i != "."]
    return digits

In [74]:
def split_conf_call_into_paras(call):
    paras_list = [t.replace("\n"," ") for t in re.split(r"\n\s*\n+", call) if len(t)>0 and not re.match(r"^\s*[0-9]+$",t)]
    return paras_list

In [75]:
def get_full_keyword(keyword, start, end, para):
    # Keyword is the first word in the paragraph
    if start == 0:
        full_keyword = keyword + " "
    # Keyword is the last word in the paragraph
    elif end == len(para) - 1:
        full_keyword = " " + keyword
    # Keyword is in the middle of a paragraph
    else:
        full_keyword = " " + keyword + " "
    return full_keyword

In [76]:
def keyword_in_paragraph(keyword, para):
    # Note that keyword must stand on its own as a word/phrase.
    # E.g. "The OCC ..." has a keyword, but "The occasion" doesn't. 
    # Idea: Check if the "full keyword" is in para, by adding appropriate spaces to the front and/or back of the keyword.
    
    # Check index of first and last char of the keyword, for every occurence of the keyword in the paragraph.
    result = re.finditer(keyword, para)
    for match_obj in result:
        start, end = match_obj.span() 
        full_keyword = get_full_keyword(keyword, start, end, para)
        # Check if full keyword is in paragraph
        if full_keyword in para:
            return True
    return False

In [77]:
def get_all_paras_containing_keywords_from_a_conf_call(row, keywords, clean = True, lower = True):
    # Split conference call into paragraphs.
    # This is an imperfect split - sometimes it breaks in the middle of a sentence
    call = str(row["Call"])
    paras_list = split_conf_call_into_paras(call)
    found_keywords, found_in_paras = [], []

    report_id = row["Report"]
    for para in paras_list:
        for keyword in keywords:
            # Lower case
            if lower == True:
                para = para.lower()
                keyword = keyword.lower()
            # Clean paragraph
            if clean == True:
                para = clean_str(para)
            # Check if keyword is in paragraph
            if keyword_in_paragraph(keyword, para):
                found_keywords.append(keyword)
                found_in_paras.append(para)
    
    # Create df 
    # Note that found_keywords and found_in_paras are lists. report_id is an int, but it will be broadcasted to a list.
    df = pd.DataFrame({"Keyword": found_keywords, "Paragraph": found_in_paras, "Report": report_id})
    return df 

In [78]:
def get_all_paras_containing_keywords_from_a_csv_file(df_csv_file):
    df_combined = pd.DataFrame()
    for _, row_conf_call in df_csv_file.iterrows():
        df = get_all_paras_containing_keywords_from_a_conf_call(row_conf_call, keywords, clean = False, lower = True)
        df_combined = pd.concat([df_combined, df])
    return df_combined

In [79]:
keywords = pd.read_csv(keywords_filepath, sep = "\t", header = None)[0]

# Main loop

In [82]:
for csv_filepath in inputfolder.iterdir():    
    csv_filename = csv_filepath.name
    print("Processing:", csv_filename)
    # Get all paragraphs containing keywords from 1 .csv file, as df_combined
    df_csv_file = pd.read_csv(csv_filepath)
    df_combined = get_all_paras_containing_keywords_from_a_csv_file(df_csv_file)
    
    # Merge paragraphs with .xls info
    unclean_data = df_combined.merge(df_csv_file, on = 'Report', how = "left")
    unclean_data["file"] = csv_filename
    unclean_data["hasnumber"] = unclean_data["Paragraph"].apply(has_numbers)
    print("Number of paras containing keywords:", unclean_data.shape[0])
    
    # Save as parquet file (to save memory)
    outputfilename = csv_filepath.stem + '.gzip'
    outputfilepath = Path(outputfolder / outputfilename)
    unclean_data.to_parquet(outputfilepath, compression = "gzip")
    print("Saved df to:", outputfilepath)
    print("---")

Processing: 20201229-20210101_1.csv
Number of paras containing keywords: 5
Saved df to: C:\Users\jasonjia\Dropbox\Projects\conference_call\output\03_identify_paragraphs_containing_keywords\20201229-20210101_1.gzip
---
Processing: 20210102-20210105_1.csv
Number of paras containing keywords: 6
Saved df to: C:\Users\jasonjia\Dropbox\Projects\conference_call\output\03_identify_paragraphs_containing_keywords\20210102-20210105_1.gzip
---
Processing: 20210106-20210109_1.csv
Number of paras containing keywords: 22
Saved df to: C:\Users\jasonjia\Dropbox\Projects\conference_call\output\03_identify_paragraphs_containing_keywords\20210106-20210109_1.gzip
---
Processing: 20210106-20210109_2.csv
Number of paras containing keywords: 18
Saved df to: C:\Users\jasonjia\Dropbox\Projects\conference_call\output\03_identify_paragraphs_containing_keywords\20210106-20210109_2.gzip
---
Processing: 20210106-20210109_3.csv
Number of paras containing keywords: 1
Saved df to: C:\Users\jasonjia\Dropbox\Projects\con

Number of paras containing keywords: 19
Saved df to: C:\Users\jasonjia\Dropbox\Projects\conference_call\output\03_identify_paragraphs_containing_keywords\20210126-20210129_5.gzip
---
Processing: 20210126-20210129_6.csv
Number of paras containing keywords: 41
Saved df to: C:\Users\jasonjia\Dropbox\Projects\conference_call\output\03_identify_paragraphs_containing_keywords\20210126-20210129_6.gzip
---
Processing: 20210126-20210129_7.csv
Number of paras containing keywords: 42
Saved df to: C:\Users\jasonjia\Dropbox\Projects\conference_call\output\03_identify_paragraphs_containing_keywords\20210126-20210129_7.gzip
---
Processing: 20210126-20210129_8.csv
Number of paras containing keywords: 36
Saved df to: C:\Users\jasonjia\Dropbox\Projects\conference_call\output\03_identify_paragraphs_containing_keywords\20210126-20210129_8.gzip
---
Processing: 20210126-20210129_9.csv
Number of paras containing keywords: 36
Saved df to: C:\Users\jasonjia\Dropbox\Projects\conference_call\output\03_identify_p

KeyboardInterrupt: 

# Tests for a single .csv file

In [23]:
csv_filepath = Path(r"C:\Users\jasonjia\Dropbox\Projects\conference_call\output\02_process_cc\02.2_csv_20210101_20220617\20201229-20210101_1.csv")

In [24]:
csv_filename = csv_filepath.name
df_csv_file = pd.read_csv(csv_filepath)
df_combined = get_all_paras_containing_keywords_from_a_csv_file(df_csv_file)
df_combined

,Keyword,Paragraph,Report
0,occ,pawan kumar goenka - mahindra & mahindra limi...,71149754
0,occ,"lawrence e. kurzius - mccormick & company, inc...",71118206
0,occ,"lawrence e. kurzius - mccormick & company, inc...",71118386
0,interest rate,"after this, we completed an $8.3 million overs...",71118254
0,occ,"following the presentation of these proposals,...",71118259
0,interest rate,"in addition, in recent years, we're also goin...",71118480
1,cost of capital,improve and achieve a return on capital which...,71118480
2,cost of capital,unidentified company representative [interpret...,71118480
0,interest rate,so apart from getting the resolution done with...,71117411
0,cost of capital,"at its core, this is a favorable recapitalizat...",71114841


In [25]:
# Merge with .xls info
unclean_data = df_combined.merge(df_csv_file, on = 'Report', how = "left")
unclean_data["file"] = csv_filename
unclean_data["hasnumber"] = unclean_data["Paragraph"].apply(has_numbers)
unclean_data

,Keyword,Paragraph,Report,PPV,TOC,Title,Subtitle,Date,Pages,Price,Contributor,Analyst,Language,Collection,Call,file,hasnumber
0,occ,pawan kumar goenka - mahindra & mahindra limi...,71149754,N,Y,MAHINDRA & MAHINDRA,MAHM.NS - Event Transcript of Mahindra and Mah...,2021-01-01,24,Subscription,THOMSON REUTERS STREETEVENTS,ANON,English,INV,\n CORPORATE PARTICIPANTS\nAnish Shah Mahindr...,20201229-20210101_1.csv,True
1,occ,"lawrence e. kurzius - mccormick & company, inc...",71118206,N,Y,MCCORMICK & CO.,MKC.N - Event Transcript of McCormick & Compan...,2020-12-30,12,Subscription,THOMSON REUTERS STREETEVENTS,ANON,English,INV,\n CORPORATE PARTICIPANTS\nKasey A. Jenkins M...,20201229-20210101_1.csv,True
2,occ,"lawrence e. kurzius - mccormick & company, inc...",71118386,N,Y,MCCORMICK & CO.,MKC.N - Event Brief of McCormick & Company Inc...,2020-12-30,15,Subscription,THOMSON REUTERS STREETEVENTS,ANON,English,INV,\n CORPORATE PARTICIPANTS\nKasey A. Jenkins M...,20201229-20210101_1.csv,True
3,interest rate,"after this, we completed an $8.3 million overs...",71118254,N,Y,HUT 8 MINING CORP,HUT.TO - Event Transcript of Hut 8 Mining Corp...,2020-12-30,9,Subscription,THOMSON REUTERS STREETEVENTS,ANON,English,INV,\n CORPORATE PARTICIPANTS\nDemetrios Vaiopoul...,20201229-20210101_1.csv,True
4,occ,"following the presentation of these proposals,...",71118259,N,Y,WPX ENERGY INC,WPX.N - Event Transcript of WPX Energy Inc con...,2020-12-30,4,Subscription,THOMSON REUTERS STREETEVENTS,ANON,English,INV,\n CORPORATE PARTICIPANTS\nRichard E. Muncrie...,20201229-20210101_1.csv,False
5,interest rate,"in addition, in recent years, we're also goin...",71118480,N,Y,UNICAJA,UNI.MC - Event Transcript of Unicaja Banco SA ...,2020-12-30,11,Subscription,THOMSON REUTERS STREETEVENTS,ANON,English,INV,\n CORPORATE PARTICIPANTS\nJaime Hernández Ma...,20201229-20210101_1.csv,False
6,cost of capital,improve and achieve a return on capital which...,71118480,N,Y,UNICAJA,UNI.MC - Event Transcript of Unicaja Banco SA ...,2020-12-30,11,Subscription,THOMSON REUTERS STREETEVENTS,ANON,English,INV,\n CORPORATE PARTICIPANTS\nJaime Hernández Ma...,20201229-20210101_1.csv,True
7,cost of capital,unidentified company representative [interpret...,71118480,N,Y,UNICAJA,UNI.MC - Event Transcript of Unicaja Banco SA ...,2020-12-30,11,Subscription,THOMSON REUTERS STREETEVENTS,ANON,English,INV,\n CORPORATE PARTICIPANTS\nJaime Hernández Ma...,20201229-20210101_1.csv,True
8,interest rate,so apart from getting the resolution done with...,71117411,N,Y,JAIN IRRIGATION SYSTEMS,JAIR.NS - Event Transcript of Jain Irrigation ...,2020-12-30,8,Subscription,THOMSON REUTERS STREETEVENTS,ANON,English,INV,\n CORPORATE PARTICIPANTS\nAnil Bhavarlal Jai...,20201229-20210101_1.csv,False
9,cost of capital,"at its core, this is a favorable recapitalizat...",71114841,N,Y,GEG.OQ - EVENT TRANSCRIPT OF FOREST INVESTMENT...,GEG.OQ - Event Transcript of Forest Investment...,2020-12-29,7,Subscription,THOMSON REUTERS STREETEVENTS,"RESEARCH DEPARTMENT, ET AL",English,INV,\n CORPORATE PARTICIPANTS\nPeter Andrew Reed ...,20201229-20210101_1.csv,False


In [30]:
# Save as parquet file
parquet_filename = csv_filepath.stem + '.gzip'
outputfilepath = Path(outputfolder / parquet_filename)
print(outputfilepath)
unclean_data.to_parquet(outputfilepath, compression = "gzip")

C:\Users\jasonjia\Dropbox\Projects\conference_call\output\03_identify_paragraphs_containing_keywords\20201229-20210101_1.gzip


# Test for keyword_in_paragraph

In [49]:
keyword = "occ"
target_string = "asd"
result = re.finditer(keyword, target_string)
for match_obj in result:
    start, end = match_obj.span() 
    print(start, end)